<a href="https://colab.research.google.com/github/RobinSmits/Dutch-LLMs/blob/main/Open_Llama_7B_Alpaca_Clean_Dutch_Qlora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
import os
from google.colab import drive
drive.mount('/content/drive')

# Set Folder to use...
WORK_DIR = '/content/drive/My Drive/LLM/OpenLlama7BAlpacaCleanDutchQlora/'
os.makedirs(WORK_DIR, exist_ok = True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q accelerate
!pip install sentencepiece
!pip install -q bitsandbytes
!pip install -q transformers
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q datasets
!pip install -q wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
import sentencepiece
from datasets import load_dataset
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType
import transformers
import wandb


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [ ]:
# WandB logging
reporting = "wandb"

# WandB Login
wandb.login()

# WandB Settings
wandb.init(project = 'OpenLlama7BAlpacaCleanDutch',
           config = {"model": 'open_llama_7B', "dataset": 'alpaca-dutch-cleaned'})

In [ ]:
# Set Name Constants
model_name = "openlm-research/open_llama_7b"
hf_model_name = 'open_llama_7b_alpaca_clean_dutch_qlora'

In [ ]:
# Create Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          use_fast = False,
                                          add_eos_token = True)
tokenizer.pad_token_id = 0

# Max Length
MAX_LEN = 512

# Tokenizer Summary
print(tokenizer)

# Special Tokens
print('pad_token_id: ',tokenizer.pad_token_id)
print('bos_token_id: ',tokenizer.bos_token_id)
print('eos_token_id: ',tokenizer.eos_token_id)
print('pad_token_id: ',tokenizer.decode(tokenizer.pad_token_id))
print('bos_token_id: ',tokenizer.decode(tokenizer.bos_token_id))
print('eos_token_id: ',tokenizer.decode(tokenizer.eos_token_id))

LlamaTokenizer(name_or_path='openlm-research/open_llama_7b', vocab_size=32000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False)
pad_token_id:  0
bos_token_id:  1
eos_token_id:  2
pad_token_id:  <unk>
bos_token_id:  <s>
eos_token_id:  </s>


In [ ]:
# Create Config
config = AutoConfig.from_pretrained(model_name,
                                    use_cache = False)

# Summary
config

LlamaConfig {
  "_name_or_path": "openlm-research/open_llama_7b",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.30.2",
  "use_cache": false,
  "vocab_size": 32000
}

In [ ]:
# Create Model
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             config = config,
                                             device_map = {"":0},
                                             quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                                                                  bnb_4bit_use_double_quant = True,
                                                                                  bnb_4bit_quant_type = 'nf4',
                                                                                  bnb_4bit_compute_dtype = torch.bfloat16))

# Enable Gradient Checkpointing
model.gradient_checkpointing_enable()

# Show Model Summary
print(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [ ]:
# Create LoRA config
loraconfig = LoraConfig(r = 8,
                        lora_alpha = 16,
                        target_modules = ['q_proj', 'v_proj'],
                        lora_dropout = 0.05,
                        bias = 'none',
                        task_type = TaskType.CAUSAL_LM)

# Prep for Training
model = prepare_model_for_kbit_training(model)

# Create LoRA Model
model = get_peft_model(model, loraconfig)
model.print_trainable_parameters()

# Show Model Summary
print(model)

trainable params: 4,194,304 || all params: 3,504,607,232 || trainable%: 0.11967971650867153
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
        

In [ ]:
# Alpaca Translated
datasets = load_dataset('BramVanroy/alpaca-cleaned-dutch')

# Summary
print(datasets)

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'instruction', 'input', 'output'],
        num_rows: 51712
    })
})


In [ ]:
# Validation size
VAL_SIZE = 2048

dataset_split = datasets["train"].train_test_split(test_size = VAL_SIZE,
                                                   shuffle = True,
                                                   seed = 42)

# Train and Val datasets
train_data = dataset_split["train"]
val_data = dataset_split["test"]

# Save for later use
val_data.save_to_disk(f'{WORK_DIR}val_data')
train_data.save_to_disk(f'{WORK_DIR}train_data')

Saving the dataset (0/1 shards):   0%|          | 0/2048 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/49664 [00:00<?, ? examples/s]

In [ ]:
def generate_prompt(data_point):
    if data_point["input"]:
        return f"""### Instructie:
{data_point["instruction"]}

### Invoer:
{data_point["input"]}

### Antwoord:
{data_point["output"]}"""
    else:
        return f"""### Instructie:
{data_point["instruction"]}

### Antwoord:
{data_point["output"]}"""

In [ ]:
counter = 0
for item_data in train_data:
    print('\n=======================================================================')
    print(generate_prompt(item_data))

    counter += 1
    if counter > 4:
        break


### Instructie:
Maak een zin met 5 uitdrukkingen.

### Antwoord:
Hij was zo boos dat hij stond te trappelen, op het verkeerde spoor zat, een appeltje te schillen had, te vroeg was begonnen, en zijn woorden moest inslikken.

### Instructie:
Voeg 3 interessante feiten over Colombia toe.

### Antwoord:
1. Colombia heeft na Brazilië de op één na grootste biodiversiteit ter wereld, met meer dan 79.000 diersoorten en 40.000 plantensoorten.
2. Colombia produceert meer koffie dan welk ander land ter wereld dan ook.
3. Colombia is het enige Zuid-Amerikaanse land met een kustlijn aan zowel de Stille Oceaan als de Caribische Zee.

### Instructie:
Geef een voorbeeld van een gebruikstoepassing voor Machine Learning

### Antwoord:
Een voorbeeld van een gebruikstoepassing voor Machine Learning is een spamfilteringsysteem. Het maakt gebruik van ML-algoritmen om spam-e-mails te detecteren en uit de inbox van een gebruiker te filteren.

### Instructie:
Herschrijf de zin om de wezelwoorden te vermijden 

In [ ]:
train_data = train_data.shuffle().map(lambda x: tokenizer.encode_plus(generate_prompt(x), truncation = True, max_length = MAX_LEN, padding = "max_length"))

Map:   0%|          | 0/49664 [00:00<?, ? examples/s]

In [ ]:
val_data = val_data.shuffle().map(lambda x: tokenizer.encode_plus(generate_prompt(x), truncation = True, max_length = MAX_LEN, padding = "max_length"))

Map:   0%|          | 0/2048 [00:00<?, ? examples/s]

In [ ]:
# Set Steps
eval_steps = 128
save_steps = 128
logging_steps = 16

# Config Trainer
trainer = transformers.Trainer(model = model,
                               train_dataset = train_data,
                               eval_dataset = val_data,
                               args = transformers.TrainingArguments(num_train_epochs = 2,
                                                                     learning_rate = 2.0e-4,
                                                                     evaluation_strategy = "steps",
                                                                     logging_steps = logging_steps,
                                                                     save_strategy = "steps",
                                                                     eval_steps = eval_steps,
                                                                     save_steps = save_steps,
                                                                     save_total_limit = 3,
                                                                     per_device_train_batch_size = 32,
                                                                     per_device_eval_batch_size = 32,
                                                                     gradient_accumulation_steps = 2,
                                                                     warmup_steps = 64,
                                                                     fp16 = True,
                                                                     output_dir = "outputs",
                                                                     optim = "paged_adamw_8bit",
                                                                     report_to = reporting),
                               data_collator = transformers.DataCollatorForLanguageModeling(tokenizer,
                                                                                            mlm = False))

# Perform Training
trainer.train()

Step,Training Loss,Validation Loss
128,1.245300,1.246110
256,1.214800,1.199918
384,1.185100,1.175393
512,1.152300,1.151609
640,1.129800,1.137925
768,1.124000,1.122694
896,1.065500,1.112084
1024,1.063300,1.102551
1152,1.051900,1.090000


Step,Training Loss,Validation Loss
128,1.245300,1.246110
256,1.214800,1.199918
384,1.185100,1.175393
512,1.152300,1.151609
640,1.129800,1.137925
768,1.124000,1.122694
896,1.065500,1.112084
1024,1.063300,1.102551
1152,1.051900,1.090000
1280,1.037900,1.080175


TrainOutput(global_step=1552, training_loss=1.124638299352115, metrics={'train_runtime': 22647.8727, 'train_samples_per_second': 4.386, 'train_steps_per_second': 0.069, 'total_flos': 1.0293857511140229e+18, 'train_loss': 1.124638299352115, 'epoch': 2.0})

In [ ]:
# Save Finetuned Model
trainer.model.save_pretrained(f'{WORK_DIR}')
tokenizer.save_pretrained(f'{WORK_DIR}')

('/content/drive/My Drive/LLM/OpenLlama7BAlpacaCleanDutchQlora/tokenizer_config.json',
 '/content/drive/My Drive/LLM/OpenLlama7BAlpacaCleanDutchQlora/special_tokens_map.json',
 '/content/drive/My Drive/LLM/OpenLlama7BAlpacaCleanDutchQlora/tokenizer.model',
 '/content/drive/My Drive/LLM/OpenLlama7BAlpacaCleanDutchQlora/added_tokens.json')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Push Model to HF Hub
model.push_to_hub(hf_model_name, private = True)

adapter_model.bin:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/robinsmits/open_llama_7b_alpaca_clean_dutch_qlora/commit/24436867e1ae131debbf9f99bbfeb08c5acac370', commit_message='Upload model', commit_description='', oid='24436867e1ae131debbf9f99bbfeb08c5acac370', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Push Tokenizer to HF Hub
tokenizer.push_to_hub(hf_model_name, private = True)

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/robinsmits/open_llama_7b_alpaca_clean_dutch_qlora/commit/3733048bef262c99ef068b2a6152002667bdcaea', commit_message='Upload tokenizer', commit_description='', oid='3733048bef262c99ef068b2a6152002667bdcaea', pr_url=None, pr_revision=None, pr_num=None)